IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M1\NO_FAULT1


In [3]:
guasto = 0

VARIABILE RCOU POTENZA MOTORI

In [4]:
#andiamo a prendere una potenza media di 1400 per considerare il momento il cui inizia/finisce il volo
rcou_m1 = pd.read_csv("RCOU.csv")
rcou_m1 = rcou_m1.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1 = rcou_m1[((rcou_m1['C9'] >= potenza) & (rcou_m1['C10'] >= potenza) & (rcou_m1['C11'] >= potenza) & (rcou_m1['C12'] >= potenza) & (rcou_m1['C13'] >= potenza) & (rcou_m1['C14']>= potenza))]

rcou_m1=rcou_m1.reset_index(drop=True)

pd.set_option('display.max_rows',700)
display(rcou_m1)

,TimeUS,C9,C10,C11,C12,C13,C14
0,22664152,1360,1375,1382,1352,1371,1363
1,22676651,1382,1369,1360,1390,1364,1387
2,22771304,1452,1411,1456,1406,1473,1386
3,22782035,1507,1358,1409,1465,1479,1393
4,22792632,1519,1355,1384,1497,1465,1420
...,...,...,...,...,...,...,...
4180,68488846,1553,1460,1514,1503,1422,1584
4181,68499144,1536,1469,1506,1501,1397,1594
4182,68509358,1542,1448,1511,1482,1419,1565
4183,68523070,1572,1386,1463,1509,1415,1550


In [5]:
min=rcou_m1['TimeUS'][0]
max=rcou_m1['TimeUS'][len(rcou_m1)-1]
print(max)
print(min)

68533079
22664152


In [6]:
#xkf utilizzato per fare poi il confronto con i valori di roll, pitch e yaw in att
xkf1_m1 = pd.read_csv("XKF1_0.csv")
xkf1_m1 = xkf1_m1.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1 = xkf1_m1[((xkf1_m1['TimeUS'] >= min) & (xkf1_m1['TimeUS'] <= max))]
xkf1_m1 = xkf1_m1.reset_index(drop=True)
print(xkf1_m1)

        TimeUS  Roll  Pitch    Yaw
0     22675051  0.01  -0.44  13.24
1     22685153  0.03  -0.40  13.24
2     22695762 -0.03  -0.45  13.23
3     22705793 -0.04  -0.52  13.22
4     22715779  0.03  -0.40  13.23
...        ...   ...    ...    ...
4256  68486359 -4.46   1.19   5.63
4257  68496743 -4.40   1.35   5.58
4258  68507280 -4.34   1.52   5.53
4259  68521132 -4.14   1.73   5.49
4260  68531134 -3.95   1.91   5.45

[4261 rows x 4 columns]


In [7]:
att_m1 = pd.read_csv("ATT.csv")
att_m1 = att_m1.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1 = att_m1[((att_m1['TimeUS'] >= min) & (att_m1['TimeUS'] <= max))]
att_m1 = att_m1.reset_index(drop=True)
print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [8]:
for idx,i in enumerate(att_m1['Roll']):
    if(xkf1_m1['Roll'][idx] != i):
        att_m1['Roll'][idx] = ((att_m1['Roll'][idx] + xkf1_m1['Roll'][idx])/2)

for idx,i in enumerate(att_m1['Pitch']):
    if(xkf1_m1['Pitch'][idx] != i):
        att_m1['Pitch'][idx] = ((att_m1['Pitch'][idx] + xkf1_m1['Pitch'][idx])/2)

for idx,i in enumerate(att_m1['Yaw']):
    if(xkf1_m1['Yaw'][idx] != i):
        att_m1['Yaw'][idx] = ((att_m1['Yaw'][idx] + xkf1_m1['Yaw'][idx])/2)

print(att_m1)

        TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835     0.00  0.03      0.00  -0.40   13.23  13.23
...        ...      ...   ...       ...    ...     ...    ...
4256  68488238    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m1 = pd.read_csv("ESC_0.csv")
esc_0_m1 = esc_0_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m1 = esc_0_m1[((esc_0_m1['TimeUS'] >= min) & (esc_0_m1['TimeUS'] <= max))]
esc_0_m1=esc_0_m1.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m1 = pd.read_csv("ESC_1.csv")
esc_1_m1 = esc_1_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m1 = esc_1_m1[((esc_1_m1['TimeUS'] >= min) & (esc_1_m1['TimeUS'] <= max))]
esc_1_m1=esc_1_m1.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m1 = pd.read_csv("ESC_2.csv")
esc_2_m1 = esc_2_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m1 = esc_2_m1[((esc_2_m1['TimeUS'] >= min) & (esc_2_m1['TimeUS'] <= max))]
esc_2_m1=esc_2_m1.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m1 = pd.read_csv("ESC_3.csv")
esc_3_m1 = esc_3_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m1 = esc_3_m1[((esc_3_m1['TimeUS'] >= min) & (esc_3_m1['TimeUS'] <= max))]
esc_3_m1=esc_3_m1.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m1 = pd.read_csv("ESC_4.csv")
esc_4_m1 = esc_4_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m1 = esc_4_m1[((esc_4_m1['TimeUS'] >= min) & (esc_4_m1['TimeUS'] <= max))]
esc_4_m1=esc_4_m1.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m1 = pd.read_csv("ESC_5.csv")
esc_5_m1 = esc_5_m1.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m1 = esc_5_m1[((esc_5_m1['TimeUS'] >= min) & (esc_5_m1['TimeUS'] <= max))]
esc_5_m1=esc_5_m1.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})



In [10]:
#valori di accelerazione e del giroscopio
imu_0_m1 = pd.read_csv("IMU_0.csv")
imu_0_m1 = imu_0_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1 = imu_0_m1[((imu_0_m1['TimeUS'] >= min) & (imu_0_m1['TimeUS'] <= max))]

imu_1_m1 = pd.read_csv("IMU_1.csv")
imu_1_m1 = imu_1_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1 = imu_1_m1[((imu_1_m1['TimeUS'] >= min) & (imu_1_m1['TimeUS'] <= max))]

imu_2_m1 = pd.read_csv("IMU_2.csv")
imu_2_m1 = imu_2_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1 = imu_2_m1[((imu_2_m1['TimeUS'] >= min) & (imu_2_m1['TimeUS'] <= max))]

imu_m1 = pd.concat((imu_0_m1, imu_1_m1, imu_2_m1))
imu_m1=imu_m1.groupby(imu_m1.TimeUS, as_index=False).mean()

display(imu_m1)


,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,22666080,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081
1,22668252,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686
2,22670635,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578
3,22675772,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946
4,22681012,-0.035033,-0.112585,-0.025552,-0.159212,0.062739,-9.546370
...,...,...,...,...,...,...,...
16503,68522130,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947
16504,68524886,0.331998,0.249299,-0.035411,-0.695731,2.937416,-12.088990
16505,68527023,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677
16506,68530131,0.314659,0.326277,-0.025974,-0.837869,3.015614,-12.519287


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m1_nofault = pd.merge_ordered(imu_m1,att_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
2      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
3      22675772  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
4      22675940  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
...         ...       ...       ...       ...       ...       ...        ...   
20764  68524886  0.331998  0.249299 -0.035411 -0.695731  2.937416 -12.088990   
20765  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
20766  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
20767  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
20768  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  Pitch  

In [12]:
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m1_nofault = pd.merge_ordered(m1_nofault, esc_0_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_1_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_2_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_3_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_4_m1)
m1_nofault = pd.merge_ordered(m1_nofault, esc_5_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
m1_nofault = pd.merge_ordered(m1_nofault, rcou_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")

print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0      22664152 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1      22666080 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2      22668252  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3      22668569  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4      22670635  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
...         ...       ...       ...       ...       ...       ...        ...   
27405  68527023  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  68530131  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  68532323  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27408  68532571  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27409  68533079  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  RP

In [13]:
#aggiornamento del tempo ponendo a 0 il momento in cui il volo inizia per noi e settaggio della variabile guasto
#m1_nofault = m1_nofault[~m1_nofault.TimeUS.isin(index_to_remove)]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"] - m1_nofault.iloc[0]["TimeUS"]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"].astype(int)
m1_nofault["Guasto"] = guasto
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1          1928 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          4100  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3          4417  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4          6483  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
...         ...       ...       ...       ...       ...       ...        ...   
27405  45862871  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
27406  45865979  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
27407  45868171  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27408  45868419  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
27409  45868927  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  Cu

In [14]:
last_cell = m1_nofault.iloc[-1, m1_nofault.columns.get_loc('TimeUS')]
print(last_cell)

45868927


In [15]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [16]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
16050  45854850
16051  45857707
16052  45860564
16053  45863421
16054  45866278

[16055 rows x 1 columns]


In [17]:
m1_nofault_final = pd.merge_ordered(m1_nofault,df)
m1_nofault_final=m1_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1          1928 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          2857 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
3          4100  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4          4417  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
...         ...       ...       ...       ...       ...       ...        ...   
43451  45865979  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
43452  45866278  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
43453  45868171  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
43454  45868419  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
43455  45868927  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  ...  Cu

In [18]:
m1_nofault_final = m1_nofault_final[m1_nofault_final.TimeUS.isin(to_be)]

In [19]:
m1_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
2,2857,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
5,5714,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
7,8571,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
8,11428,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43442,45854850,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
43443,45857707,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
43447,45860564,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
43450,45863421,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0


In [20]:
from datetime import timedelta
m1_nofault_final=m1_nofault_final.reset_index(drop=True)
m1_nofault_final['TimeUS'] = pd.to_datetime(m1_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
1,00:00:00.002857,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
2,00:00:00.005714,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
3,00:00:00.008571,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
4,00:00:00.011428,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,...,1.90,3348.0,1.99,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16050,00:00:45.854850,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
16051,00:00:45.857707,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,...,1.42,5687.0,3.29,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
16052,00:00:45.860564,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
16053,00:00:45.863421,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677,-1.39,-4.14,-0.18,...,1.42,5687.0,3.29,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0


## Analisi nel tempo e in frequenza

In [21]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_nofault_final)/350)):
        V1=m1_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab
    #display(tab)

In [23]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
#result = pd.merge(, how="right")
display(df_merged)


d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_11224\3363470071.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_11224\3363470071.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.041897,0.019825,-0.908719,0.146711,69.0,51.0,6.981244,3.069249,0.016826,...,10479.649542,3.231481,0.540083,-1.086018,3.313762,109.0,108.0,41.309706,7.079391,0
1,1.0,-0.042134,0.019788,-0.909437,0.146654,81.0,60.0,7.061480,3.105682,0.016689,...,10713.204087,3.233750,0.540356,-1.081089,3.316016,101.0,100.0,41.412183,7.172989,0
2,2.0,-0.042387,0.019755,-0.910883,0.146614,71.0,52.0,7.123181,3.241375,0.016595,...,10913.900128,3.236006,0.540617,-1.076148,3.318255,105.0,104.0,41.490669,7.221693,0
3,3.0,-0.042453,0.019700,-0.906016,0.146448,73.0,56.0,7.390245,3.190626,0.016499,...,10779.487685,3.238249,0.540866,-1.071195,3.320481,105.0,104.0,41.545321,7.226421,0
4,4.0,-0.042560,0.019649,-0.902198,0.146304,71.0,55.0,7.642152,3.048569,0.016327,...,10551.335879,3.240479,0.541104,-1.066231,3.322692,107.0,106.0,41.576309,7.188369,0
5,5.0,-0.042775,0.019610,-0.902065,0.146234,61.0,47.0,7.850293,2.989178,0.016172,...,10305.353511,3.242697,0.541331,-1.061254,3.324890,105.0,103.0,41.583818,7.114804,0
6,6.0,-0.042939,0.019565,-0.900069,0.146128,77.0,58.0,8.069948,3.173532,0.016085,...,10042.484703,3.244902,0.541546,-1.056264,3.327074,107.0,105.0,41.568048,7.245697,0
7,7.0,-0.042940,0.019510,-0.894137,0.145942,83.0,61.0,8.342013,3.336372,0.015934,...,9763.716608,3.247095,0.541751,-1.051262,3.329244,105.0,103.0,41.529211,7.379113,0
8,8.0,-0.043115,0.019466,-0.892529,0.145845,75.0,55.0,8.545529,3.033171,0.015757,...,9531.990647,3.249276,0.541945,-1.046247,3.331400,111.0,109.0,41.467533,7.507921,0
9,9.0,-0.043262,0.019420,-0.889959,0.145730,75.0,56.0,8.752447,2.961937,0.015717,...,9508.413274,3.251444,0.542129,-1.041219,3.333544,109.0,107.0,41.383252,7.624827,0


In [24]:
path_file = path_file.replace(r"M1\NO_FAULT1", "")
os.chdir(path_file)
df_merged.to_csv('m1_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
